In [1]:

import os
fname_prefix = "/data/users/team2_capstone/code-style-probing/"
os.environ["CUDA_VISIBLE_DEVICES"] = '5'
import numpy as np
import pandas as pd
#import pandas as pd
from transformers import *
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from datasets import load_from_disk, load_metric
#import datasets
import pickle
#from utils.helper import read_py150k_code, read_file_to_string
import regex as re

In [2]:
tokenizer = RobertaTokenizer.from_pretrained('Salesforce/codet5-small')
"""train_codet5_dataset = load_from_disk(fname_prefix + "datasets/codet5_train_docstring_bq_padded.hf")
id = 70000
dups = 0
print (len(train_codet5_dataset))
for id in range(0,1000):
    #input = (tokenizer.decode(train_codet5_dataset[id* 100]['input_ids'], skip_special_tokens=True))
    #label = (tokenizer.decode(train_codet5_dataset[id* 100]['labels'], skip_special_tokens=True))
    input = (train_codet5_dataset[id* 100]['input_ids'])
    label = (train_codet5_dataset[id* 100]['labels'])
       if "\"\"\"" in input:
        print ("============================")
        print (input)
        print("++++++++++++++")
        print (label)
    if input == label:
        dups +=1
print (dups)"""

loading file https://huggingface.co/Salesforce/codet5-small/resolve/main/vocab.json from cache at /soe/ksmunson/.cache/huggingface/transformers/12dd1d7e87ade1728f8382f3b875a47cdddb88bc50797ecc506957411661a39a.9a48c5abf25554713c6513ab01066e53569b9a2da0d6189715951cf7c6288805
loading file https://huggingface.co/Salesforce/codet5-small/resolve/main/merges.txt from cache at /soe/ksmunson/.cache/huggingface/transformers/c2ada6c76bb6a90c2330323775cb4853dd2b0cfee29d6b2c5ecb419c5874b488.7179059568f1a130b0a79e4bac71f38545207cab0ec45ce82ca09afadb2649a3
loading file https://huggingface.co/Salesforce/codet5-small/resolve/main/added_tokens.json from cache at /soe/ksmunson/.cache/huggingface/transformers/354ae288bdee65437fa8eedecf9a2770001b97bac23d7fc5a04badae8da42346.5cc6e825eb228a7a5cfd27cb4d7151e97a79fb962b31aaf1813aa102e746584b
loading file https://huggingface.co/Salesforce/codet5-small/resolve/main/special_tokens_map.json from cache at /soe/ksmunson/.cache/huggingface/transformers/f420b33feb234c

'train_codet5_dataset = load_from_disk(fname_prefix + "datasets/codet5_train_docstring_bq_padded.hf")\nid = 70000\ndups = 0\nprint (len(train_codet5_dataset))\nfor id in range(0,1000):\n    #input = (tokenizer.decode(train_codet5_dataset[id* 100][\'input_ids\'], skip_special_tokens=True))\n    #label = (tokenizer.decode(train_codet5_dataset[id* 100][\'labels\'], skip_special_tokens=True))\n    input = (train_codet5_dataset[id* 100][\'input_ids\'])\n    label = (train_codet5_dataset[id* 100][\'labels\'])\n       if """"" in input:\n        print ("============================")\n        print (input)\n        print("++++++++++++++")\n        print (label)\n    if input == label:\n        dups +=1\nprint (dups)'

In [3]:


#model = T5ForConditionalGeneration.from_pretrained('Salesforce/codet5-base')
train_codet5_dataset = load_from_disk(fname_prefix + 'datasets/codet5_train_docstring_bq_padded.hf')
test_codet5_dataset = load_from_disk(fname_prefix + 'datasets/codet5_test_docstring_bq_padded.hf') #codet5_train_class_bq_padded.hf

"""def replaceNegative(label):
    #print (label['labels'])

    label['labels'] = [label['labels']]
    return label

train_codet5_dataset = train_codet5_dataset.map(replaceNegative, batched = False)
test_codet5_dataset = test_codet5_dataset.map(replaceNegative, batched = False)"""

"def replaceNegative(label):\n    #print (label['labels'])\n\n    label['labels'] = [label['labels']]\n    return label\n\ntrain_codet5_dataset = train_codet5_dataset.map(replaceNegative, batched = False)\ntest_codet5_dataset = test_codet5_dataset.map(replaceNegative, batched = False)"

In [4]:
model_checkpoint = "Salesforce/codet5-small"
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
eval_dataset = test_codet5_dataset
eval_dataset = eval_dataset.train_test_split(test_size = 0.08)
print (eval_dataset)

loading configuration file https://huggingface.co/Salesforce/codet5-small/resolve/main/config.json from cache at /soe/ksmunson/.cache/huggingface/transformers/ef13e715cbf36adda46c74774e8032ab573cfbb2ebe59748c9fc72b7cf67e418.96d28e790b8c3d3e3be663606a66e0793a173c78e745e3603be4c0f878319099
Model config T5Config {
  "_name_or_path": "Salesforce/codet5-small",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "bos_token_id": 1,
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 512,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 2,
  "feed_forward_proj": "relu",
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 6,
  "num_heads": 8,
  "num_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
 

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0.1', 'Unnamed: 0', 'repository', 'filepath', 'forks', 'issue_events', 'stars', 'parse_error', 'line_count', 'comment_count', 'comment_avg_len', 'comment_density', 'id_total', 'lower_case', 'id_total_var', 'lower_case_var', 'snake_case', 'snake_case_var', 'lower_camel_case', 'lower_camel_case_var', 'snake_case_ratio', 'snake_case_var_ratio', 'snake_case_class_ratio', 'snake_case_method_ratio', 'lower_camel_case_ratio', 'lower_camel_case_var_ratio', 'lower_camel_case_class_ratio', 'lower_camel_case_method_ratio', 'upper_camel_case_ratio', 'upper_camel_case_var_ratio', 'upper_camel_case_class_ratio', 'upper_camel_case_method_ratio', 'lower_case_ratio', 'lower_case_var_ratio', 'lower_case_class_ratio', 'lower_case_method_ratio', 'upper_case_ratio', 'upper_case_var_ratio', 'upper_case_class_ratio', 'upper_case_method_ratio', 'other_case_ratio', 'other_case_var_ratio', 'other_case_class_ratio', 'other_case_method_ratio', 'func_

In [5]:
batch_size = 24
model_name = model_checkpoint.split("/")[-1]
from transformers import AutoConfig

#config = AutoConfig.from_pretrained(fname_prefix + "/seq2seq_results/no_outlier_codet5small/checkpoint-51000",    min_length = 20, max_length = 512, )
#model =AutoModelForSeq2SeqLM.from_pretrained(fname_prefix + "/seq2seq_results/outlier_codet5small/checkpoint-40500")
#model =AutoModelForSeq2SeqLM.from_pretrained("Salesforce/codet5-small")
model =AutoModelForSeq2SeqLM.from_pretrained(fname_prefix + "/seq2seq_results/outlier_docstring_codet5small/checkpoint-46000")
model.config.max_length = 512
#fname_prefix + /seq2seq_results/no_outlier_codet5small/checkpoint-50500
args = Seq2SeqTrainingArguments(
    #f"{model_name}-finetuned-xsum",
    output_dir= fname_prefix + "/seq2seq_results/outlier_docstring_codet5small",
    evaluation_strategy='epoch',
    #eval_steps=20000,
    eval_accumulation_steps=200,
    learning_rate=2e-3,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=30,
    num_train_epochs=4,
    predict_with_generate=True,
    push_to_hub=False,

)

#data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

loading configuration file /data/users/team2_capstone/code-style-probing//seq2seq_results/outlier_docstring_codet5small/checkpoint-46000/config.json
Model config T5Config {
  "_name_or_path": "/data/users/team2_capstone/code-style-probing//seq2seq_results/outlier_docstring_codet5small/checkpoint-46000",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "bos_token_id": 1,
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 512,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 2,
  "feed_forward_proj": "relu",
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_epsilon": 1e-06,
  "max_length": 512,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 6,
  "num_heads": 8,
  "num_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance

In [6]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=train_codet5_dataset ,
    eval_dataset=test_codet5_dataset,
    #data_collator=data_collator,
    tokenizer=tokenizer,
    
)
#model.generate()
#
trainer.train()

The following columns in the training set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: upper_camel_case_var, __index_level_0__, id_total_method, snake_case_method, Unnamed: 0.1, other_case, id_total_class, comment_density, func_decorators_avg, ds_word_len_avg, other_case_var, lower_case_class_ratio, uncommented_content, lower_camel_case_ratio, ds_line_count, class_decorators_count, content, func_async_ratio, lower_case_class, lower_camel_case_method, snake_case, issue_events, snake_case_var_ratio, upper_camel_case_class, upper_case_var, func_count, ds_char_len_avg, upper_case_class, overridden_method, file, other_case_var_ratio, upper_case, lower_camel_case_class_ratio, other_case_class, other_case_method, comment_avg_len, lower_case_ratio, comment_total_len, filepath, upper_camel_case_method, class_decorators_avg, upper_case_var_ratio, lambda_avg, upper_case_method_ratio, generators, other_case_method_ratio, comment_count, id_total

Epoch,Training Loss,Validation Loss
1,0.508000,0.484005
2,0.478900,0.462938
3,0.454300,0.444800
4,0.432300,0.433287


Saving model checkpoint to /data/users/team2_capstone/code-style-probing//seq2seq_results/outlier_docstring_codet5small/checkpoint-500
Configuration saved in /data/users/team2_capstone/code-style-probing//seq2seq_results/outlier_docstring_codet5small/checkpoint-500/config.json
Model weights saved in /data/users/team2_capstone/code-style-probing//seq2seq_results/outlier_docstring_codet5small/checkpoint-500/pytorch_model.bin
tokenizer config file saved in /data/users/team2_capstone/code-style-probing//seq2seq_results/outlier_docstring_codet5small/checkpoint-500/tokenizer_config.json
Special tokens file saved in /data/users/team2_capstone/code-style-probing//seq2seq_results/outlier_docstring_codet5small/checkpoint-500/special_tokens_map.json
Deleting older checkpoint [/data/users/team2_capstone/code-style-probing/seq2seq_results/outlier_docstring_codet5small/checkpoint-31500] due to args.save_total_limit
Saving model checkpoint to /data/users/team2_capstone/code-style-probing//seq2seq_res

TrainOutput(global_step=46728, training_loss=0.4758622658169039, metrics={'train_runtime': 27408.2378, 'train_samples_per_second': 40.915, 'train_steps_per_second': 1.705, 'total_flos': 1.5177446163952435e+17, 'train_loss': 0.4758622658169039, 'epoch': 4.0})

In [7]:
eval_preds = trainer.predict(eval_dataset['test']) #, min_length=40, max_length = 512)


The following columns in the test set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: upper_camel_case_var, __index_level_0__, id_total_method, snake_case_method, Unnamed: 0.1, other_case, id_total_class, comment_density, func_decorators_avg, ds_word_len_avg, other_case_var, lower_case_class_ratio, uncommented_content, lower_camel_case_ratio, ds_line_count, class_decorators_count, content, func_async_ratio, lower_case_class, lower_camel_case_method, snake_case, issue_events, snake_case_var_ratio, upper_camel_case_class, upper_case_var, func_count, ds_char_len_avg, upper_case_class, overridden_method, file, other_case_var_ratio, upper_case, lower_camel_case_class_ratio, other_case_class, other_case_method, comment_avg_len, lower_case_ratio, comment_total_len, filepath, upper_camel_case_method, class_decorators_avg, upper_case_var_ratio, lambda_avg, upper_case_method_ratio, generators, other_case_method_ratio, comment_count, id_total_var

In [8]:
predictions, labels, report  = eval_preds
decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
df = pd.DataFrame(decoded_preds,  columns = ["preds"])
df['labels'] = decoded_labels
#print (decoded_labels[0])
df.to_csv(fname_prefix + 'seq2seq_results/outlier_docstring_codet5small/codet5_preds.csv')

In [9]:



"""idx = 30
metric = load_metric("bleu") #this doesnt work for some reason
#metric = load_metric()
def compute_metrics(eval_preds):
    predictions, labels, report  = eval_preds
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    df = pd.DataFrame(decoded_preds,  columns = ["preds"])
    df['labels'] = decoded_labels
    #print (decoded_labels[0])
    df.to_csv(fname_prefix + 'seq2seq_results/no_outlier_codet5small/codet5_preds.csv')
    return metric.compute(predictions=decoded_preds, references=decoded_labels)

compute_metrics(eval_preds)"""

'idx = 30\nmetric = load_metric("bleu") #this doesnt work for some reason\n#metric = load_metric()\ndef compute_metrics(eval_preds):\n    predictions, labels, report  = eval_preds\n    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)\n    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)\n    df = pd.DataFrame(decoded_preds,  columns = ["preds"])\n    df[\'labels\'] = decoded_labels\n    #print (decoded_labels[0])\n    df.to_csv(fname_prefix + \'seq2seq_results/no_outlier_codet5small/codet5_preds.csv\')\n    return metric.compute(predictions=decoded_preds, references=decoded_labels)\n\ncompute_metrics(eval_preds)'